In [1]:
%env WANDB_PROJECT=aspect_explain_impact

env: WANDB_PROJECT=aspect_explain_impact


In [2]:
from project_dataset import load_dataset


In [3]:
from dataclasses import dataclass

@dataclass
class Args:
    model_name = "Salesforce/codet5p-220m"
    num_proc = 8
    batch_size = 5
    max_src_length = 1200
    max_des_length = 167
    data_cols = ["CVE ID", "explain", "func_before"]
    save_dir = 'tf_board'
    epochs = 100
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "impact"
    
args = Args()

In [4]:
ds = load_dataset(args.task)

In [5]:
import os

os.makedirs(args.save_dir, exist_ok=True)

In [6]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [7]:
import numpy as np

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()
    # lables = np.where(lables != codet5p_tokenizer.pad_token_id, lables, -100)

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = ds.map(
  preprocess_function,
  remove_columns=args.data_cols,
  batched=True,
  num_proc=args.num_proc,
  batch_size=args.batch_size)

tokenized_ds

Map (num_proc=8):   0%|          | 0/7032 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/782 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1954 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7032
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 782
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1954
    })
})

In [8]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name).to(device)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=model,
  return_tensors="pt")

In [10]:
import evaluate
import numpy as np

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = codet5p_tokenizer(arg)
  return codet5p_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, codet5p_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = codet5p_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = codet5p_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  text_preds = ["\n".join(p for p in text_preds)]
  text_labels = ["\n".join(l for l in text_labels)]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [11]:
# for testing, show first 5 rows

from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()
  break

metrics_func([preds, labels])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Token indices sequence length is longer than the specified maximum sequence length for this model (832 > 512). Running this sequence through the model will result in indexing errors


{'rouge1': 0.3202054794520548,
 'rouge2': 0.1269296740994854,
 'rougeL': 0.16952054794520546,
 'rougeLsum': 0.3112290008841733}

In [12]:
preds.shape, labels.shape

(torch.Size([5, 167]), torch.Size([5, 167]))

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        report_to='wandb',
        output_dir=args.save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',
        do_eval=True,

        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_acc_steps,

        learning_rate=args.lr,
        weight_decay=args.weight_decay,
        warmup_steps=args.lr_warmup_steps,

        logging_dir=args.save_dir,
        logging_first_step=True,
        logging_steps=args.log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=args.num_proc,

        local_rank=args.local_rank,
        deepspeed=args.deepspeed,
        fp16=args.fp16,
    )

In [14]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    tokenizer = codet5p_tokenizer,
)

trainer.train()

/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dongchirua. Use `wandb login --relogin` to force relogin


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,8.958000
10,8.726900
20,7.562300
30,6.227600
40,5.811100
50,5.427900
60,5.060100
70,4.619100
80,4.247300
90,4.018000


/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/hqn650/anaconda3/envs/vul-intext-reason/lib/python3.11/site-package

TrainOutput(global_step=17500, training_loss=0.06792154432581406, metrics={'train_runtime': 52639.1284, 'train_samples_per_second': 13.359, 'train_steps_per_second': 0.332, 'total_flos': 9.990715392e+17, 'train_loss': 0.06792154432581406, 'epoch': 99.57})

In [15]:
trainer.save_model(f'{args.save_dir}/{args.task}')

## measure on test set

In [16]:
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=args.batch_size,
  num_workers=args.num_proc)

rouge_list = []

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()

  rouge_result = metrics_func([preds, labels])
  print(rouge_result)
  rouge_list.append(rouge_result)

{'rouge1': 0.9298245614035087, 'rouge2': 0.8211143695014662, 'rougeL': 0.8596491228070176, 'rougeLsum': 0.9075144508670521}
{'rouge1': 0.9121212121212121, 'rouge2': 0.8601823708206686, 'rougeL': 0.8757575757575758, 'rougeLsum': 0.9131736526946107}
{'rouge1': 0.7773049645390071, 'rouge2': 0.6600284495021338, 'rougeL': 0.6666666666666666, 'rougeLsum': 0.7601683029453015}
{'rouge1': 0.8215488215488215, 'rouge2': 0.7398648648648649, 'rougeL': 0.7003367003367004, 'rougeLsum': 0.8239202657807309}
{'rouge1': 0.8747152619589978, 'rouge2': 0.8146453089244852, 'rougeL': 0.774487471526196, 'rougeLsum': 0.8769574944071589}
{'rouge1': 0.89, 'rouge2': 0.7437185929648242, 'rougeL': 0.78, 'rougeLsum': 0.8823529411764707}
{'rouge1': 0.68075117370892, 'rouge2': 0.4245283018867924, 'rougeL': 0.4507042253521127, 'rougeLsum': 0.6497695852534563}
{'rouge1': 0.863849765258216, 'rouge2': 0.6415094339622642, 'rougeL': 0.6525821596244131, 'rougeLsum': 0.8387096774193548}
{'rouge1': 0.9158878504672896, 'rouge2':

{'rouge1': 0.9324324324324323, 'rouge2': 0.823728813559322, 'rougeL': 0.8445945945945947, 'rougeLsum': 0.9166666666666666}
{'rouge1': 0.8990228013029316, 'rouge2': 0.8692810457516339, 'rougeL': 0.8664495114006515, 'rougeLsum': 0.9003215434083601}
{'rouge1': 0.908284023668639, 'rouge2': 0.7121661721068249, 'rougeL': 0.6775147928994083, 'rougeLsum': 0.8567251461988304}
{'rouge1': 0.9397590361445782, 'rouge2': 0.837772397094431, 'rougeL': 0.7903614457831326, 'rougeLsum': 0.9361702127659575}
{'rouge1': 0.8690228690228691, 'rouge2': 0.7139874739039667, 'rougeL': 0.6694386694386695, 'rougeLsum': 0.8466257668711658}
{'rouge1': 0.8918322295805738, 'rouge2': 0.5764966740576496, 'rougeL': 0.6357615894039735, 'rougeLsum': 0.8373101952277656}
{'rouge1': 0.971311475409836, 'rouge2': 0.9629629629629629, 'rougeL': 0.971311475409836, 'rougeLsum': 0.9717741935483872}
{'rouge1': 0.911465892597968, 'rouge2': 0.7714701601164483, 'rougeL': 0.702467343976778, 'rougeLsum': 0.8837876614060257}
{'rouge1': 0.82

In [17]:
print(codet5p_tokenizer.batch_decode(preds[0], skip_special_tokens=True))

['', '', 'c', ' a', ' u', ' s', ' e', '  ', ' a', '  ', ' d', ' e', ' n', ' i', ' a', ' l', '  ', ' o', ' f', '  ', ' s', ' e', ' r', ' v', ' i', ' c', ' e', '  ', ' (', ' a', ' p', ' p', ' l', ' i', ' c', ' a', ' t', ' i', ' o', ' n', '  ', ' c', ' r', ' a', ' s', ' h', '  ', ' )', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [18]:
print(codet5p_tokenizer.batch_decode(labels[0], skip_special_tokens=True))

['', 'c', ' a', ' u', ' s', ' e', '  ', ' a', '  ', ' d', ' e', ' n', ' i', ' a', ' l', ' -', ' o', ' f', ' -', ' s', ' e', ' r', ' v', ' i', ' c', ' e', '  ', ' (', ' u', ' n', ' c', ' o', ' n', ' t', ' r', ' o', ' l', ' l', ' e', ' d', '  ', ' r', ' e', ' s', ' o', ' u', ' r', ' c', ' e', '  ', ' c', ' o', ' n', ' s', ' u', ' m', ' p', ' t', ' i', ' o', ' n', '  ', ' )', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [19]:
rouge1_values = [d['rouge1'] for d in rouge_list]
rouge2_values = [d['rouge2'] for d in rouge_list]
rougeL_values = [d['rougeL'] for d in rouge_list]
rouge1_values = sum(rouge1_values) / len(rougeL_values)
average_rouge2 = sum(rouge2_values) / len(rougeL_values)
average_rougeL = sum(rougeL_values) / len(rougeL_values)

rouge1_values, average_rouge2, average_rougeL

(0.8888556584360371, 0.7716924286035672, 0.7580520686170895)